<a href="https://colab.research.google.com/github/jikhans/Parallel-Computing-with-Python-and-R/blob/master/_Oct_4%2C_2019_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Oct 4, 2019 Dask
* name: Jikhan Jeong
* ref: https://towardsdatascience.com/trying-out-dask-dataframes-in-python-for-fast-data-analysis-in-parallel-aa960c18a915
* ref: github: https://github.com/StrikingLoo/dask-dataframe-benchmarking

* Arithmetic operations (multiplying or adding to a Series)
* Common aggregations (mean, min, max, sum, etc.)
* Calling apply (as long as it’s along the index -that is, not after a 
* groupby(‘y’) where ‘y’ is not the index-)
* Calling value_counts(), drop_duplicates() or corr()
* Filtering with loc, isin, and row-wise selection

In [0]:
import dask.dataframe as ddf
import time
import pandas as pd

# Dask Dataframe

In [0]:
df = ddf.read_csv('random_people.csv')
df['bonus'] = df['salary']*.5
df = df.drop('Unnamed: 0', axis=1)

In [10]:
df.head(2)

,name,surname,salary,bonus
0,Henry,Joneson,5000,2500.0
1,Albert,Goodman,10000,5000.0


# Pandas Dataframe

In [0]:
df_old = pd.read_csv('random_people.csv')

In [0]:
df_old['bonus'] = df_old['salary']*.5
df_old = df_old.drop('Unnamed: 0', axis=1)

In [13]:
df_old.head(2)

,name,surname,salary,bonus
0,Henry,Joneson,5000,2500.0
1,Albert,Goodman,10000,5000.0


In [0]:
def benchmark(function, function_name):
    start = time.time()
    function()
    end = time.time()
    print("{0} seconds for {1}".format((end - start), function_name))

In [0]:
def get_bonus(df):
    df['bonus'] = df['salary']*.7

In [0]:
def test_1():
    get_bonus(df) # dask 
def test_2():
    get_bonus(df_old) # non-dask

In [17]:
benchmark(test_1, 'dataframe with dask')
benchmark(test_2, 'dataframe without dask')

0.004058837890625 seconds for dataframe with dask
0.0005838871002197266 seconds for dataframe without dask


In [0]:
df_old2 = pd.concat([df_old for _ in range(1000)])
df_old3 = pd.concat([df_old2 for _ in range(500)])

In [20]:
df_old3.shape

(25000000, 4)

# Check the number of cores

In [23]:
import multiprocessing
multiprocessing.cpu_count()

2

# Transform Pandas Dataframe to Dask Dataframe

In [0]:
ddf_big = ddf.from_pandas(df_old3, npartitions=2) # number of partition = # of core

# Compare the performance
* Pandas big dataframe : df_old3
* Dask big data frame L ddf_big

In [0]:
def test_big():
    get_bonus(ddf_big)
def test_big_old():
    get_bonus(df_old3)

def get_big_mean():
    return ddf_big.salary.mean().compute()
def get_big_mean_old():
    return  df_old3.salary.mean()

def get_big_max():
    return ddf_big.salary.max().compute()
def get_big_max_old():
    return  df_old3.salary.max()

def get_big_sum():
    return ddf_big.salary.sum().compute()
def get_big_sum_old():
    return  df_old3.salary.sum()

def filter_df():
    df = ddf_big[ddf_big['salary']>5000]
def filter_df_old():
    df =  df_old3[ df_old3['salary']>5000]
    
def run_benchmarks():
    for i,f in enumerate([test_big, #test_big_old,
                          get_big_mean,# get_big_mean_old,
                          get_big_max, #get_big_max_old,
                          get_big_sum, #get_big_sum_old,
                          filter_df,#filter_df_old
                         ]):
        benchmark(f, f.__name__)

In [0]:
def f(x):
    return (13*x+5)%7

def apply_random_old():
    df_old3['random']= df_old3['salary'].apply(f)
    
def apply_random():
    ddf_big['random']= ddf_big['salary'].apply(f).compute()

def value_count_test():
    ddf_big.salary.value_counts().compute()

def value_count_test_old():
    df_old3.salary.value_counts()

# Dask Performance

In [28]:
run_benchmarks()

0.006788492202758789 seconds for test_big
0.7481985092163086 seconds for get_big_mean
0.6630303859710693 seconds for get_big_max
0.6824014186859131 seconds for get_big_sum
0.0032804012298583984 seconds for filter_df


# Dask

In [29]:
benchmark(apply_random, apply_random.__name__)

/usr/local/lib/python3.6/dist-packages/dask/dataframe/core.py:2294: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('salary', 'int64'))

  warnings.warn(meta_warning(meta))


13.085474967956543 seconds for apply_random


In [33]:
benchmark(value_count_test, value_count_test.__name__)

1.3808255195617676 seconds for value_count_test


# Pandas

In [30]:
benchmark(apply_random_old, apply_random_old.__name__)

10.283009767532349 seconds for apply_random_old


In [32]:
benchmark(value_count_test_old, value_count_test_old.__name__)

0.2420790195465088 seconds for value_count_test_old
